In [1]:
from config import *
config_chapterextra()
# This is needed to render the plots in this chapter
from plots.chapterextra import *


In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.datasets import make_regression

from torch.utils.data import DataLoader,TensorDataset,Dataset
from stepbystep.v3 import StepByStep

from data_generation.ball import load_data


2024-03-20 11:02:25.281347: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 11:02:25.326863: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 11:02:25.326896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 11:02:25.327986: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-20 11:02:25.334671: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 11:02:25.335498: I tensorflow/core/platform/cpu_feature_guard.cc:1

As the model grow deeper, gradient computed for weights at initial layers become smaller and smaller. its called `vanishing gradients`

## Ball Dataset and Block Model

In [3]:
# Data generation
X,y=load_data(n_points=1000,n_dims=10)


In [4]:
# Data preparation
ball_dataset=TensorDataset(torch.as_tensor(X,dtype=torch.float32),torch.as_tensor(y,dtype=torch.float32))

ball_loader=DataLoader(ball_dataset,batch_size=(len(X)))

In [5]:
# def build_model(n_features,n_layers,hidden_units,activation_fn,use_bn=False):

#     layers=nn.Sequential()
#     # input layer
#     layers.add_module("h1",nn.Linear(n_features,hidden_units))
    
#     # hidden layers
#     for i in range(n_layers-1):
#         layers.add_module(f"a{i+1}",activation_fn)
#         if use_bn:
#             layers.add_module(f"bn{i+1}",nn.BatchNorm1d(hidden_units))
#         layers.add_module(f"h{i+2}",nn.Linear(hidden_units,hidden_units))
    
#     # output layer
#     layers.add_module(f"a{i+2}",activation_fn)
#     if use_bn:
#         layers.add_module(f"bn{i+2}",nn.BatchNorm1d(hidden_units))
#     layers.add_module("o",nn.Linear(hidden_units,1))
#     return layers
    




In [6]:
# Model  configuration
torch.manual_seed(11)
n_features=X.shape[1]
n_layers=5
hidden_units=100
activation_fn=nn.ReLU()
model=build_model(n_features,n_layers,hidden_units,activation_fn,use_bn=False)

TypeError: ReLU.forward() missing 1 required positional argument: 'input'

In [ ]:
print(model)

In [ ]:
# Model configuration
loss_fn=nn.BCEWithLogitsLoss()
optimizer=optim.SGD(model.parameters(),lr=1e-2)


### Weights, Acivations and Gradients

In [ ]:
# Model training
hidden_layers=[f'h{i}' for i in range(1,n_layers+1)]
activation_layers=[f'a{i}' for i in range(1,n_layers+1)]

sbs=StepByStep(model,loss_fn,optimizer)
sbs.set_loaders(ball_loader)

sbs.capture_parameters(hidden_layers)
sbs.capture_gradients(hidden_layers)
sbs.attach_hooks(activation_layers)
sbs.train(1)


In [ ]:
params,gradinets,activations=get_plot_data(train_loader=ball_loader,model=model)